<a href="https://colab.research.google.com/github/shankar-tester901/cv/blob/main/ASLwithCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/kumarvivek9088/aslsigndataset.git


Cloning into 'aslsigndataset'...
remote: Enumerating objects: 6673, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 6673 (delta 0), reused 0 (delta 0), pack-reused 6670
Receiving objects: 100% (6673/6673), 151.97 MiB | 28.02 MiB/s, done.
Resolving deltas: 100% (14/14), done.


In [2]:
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard
import os


In [3]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
)

val_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 128

train_generator = train_datagen.flow_from_directory(
    '/content/aslsigndataset/splitdataset48x48/train',
    target_size=(48, 48),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='grayscale'
)

validation_generator = val_datagen.flow_from_directory(
    '/content/aslsigndataset/splitdataset48x48/val',
    target_size=(48, 48),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='grayscale'
)

Found 1473 images belonging to 6 classes.
Found 372 images belonging to 6 classes.


In [4]:
class_names = list(train_generator.class_indices.keys())
print(class_names)

['A', 'M', 'N', 'S', 'T', 'blank']


In [5]:

model = Sequential()
# convolutional layers
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
# fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
# output layer
model.add(Dense(6, activation='softmax'))

In [6]:
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 128)       1280      
                                                                 
 max_pooling2d (MaxPooling2  (None, 23, 23, 128)       0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 23, 23, 128)       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 21, 21, 256)       295168    
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 10, 10, 256)       0         
 g2D)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 10, 10, 256)       0

In [7]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = 'accuracy' )


In [8]:

!rm -rf Logs
logdir = os.path.join("Logs")
tensorboard_callback = TensorBoard(log_dir=logdir)

In [ ]:

model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=100,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    callbacks=[tensorboard_callback]
)

Epoch 1/100
11/11 [==============================] - 73s 6s/step - loss: 1.7940 - accuracy: 0.1799 - val_loss: 1.7861 - val_accuracy: 0.1875
Epoch 2/100
11/11 [==============================] - 68s 6s/step - loss: 1.7871 - accuracy: 0.1888 - val_loss: 1.7855 - val_accuracy: 0.1719
Epoch 3/100
11/11 [==============================] - 68s 6s/step - loss: 1.7794 - accuracy: 0.2007 - val_loss: 1.7687 - val_accuracy: 0.1758
Epoch 4/100
11/11 [==============================] - 68s 6s/step - loss: 1.7621 - accuracy: 0.1829 - val_loss: 1.7258 - val_accuracy: 0.1953
Epoch 5/100
11/11 [==============================] - 67s 6s/step - loss: 1.7258 - accuracy: 0.1955 - val_loss: 1.7114 - val_accuracy: 0.2109
Epoch 6/100
11/11 [==============================] - 67s 6s/step - loss: 1.7075 - accuracy: 0.2037 - val_loss: 1.6602 - val_accuracy: 0.2148
Epoch 7/100
11/11 [==============================] - 67s 6s/step - loss: 1.6644 - accuracy: 0.2327 - val_loss: 1.6437 - val_accuracy: 0.2656
Epoch 8/100
1

In [ ]:
%load_ext tensorboard
%tensorboard --logdir Logs

In [ ]:

model_json = model.to_json()
with open("/content/drive/MyDrive/signlanguagedetectionmodel48x48.json",'w') as json_file:
    json_file.write(model_json)
model.save("/content/drive/MyDrive/signlanguagedetectionmodel48x48.h5")